# Task 8 — Hyperparameter Tuning (quick demo using Diabetes dataset)

This notebook demonstrates hyperparameter tuning on a Random Forest for regression using sklearn's Diabetes dataset. The tuning run is intentionally small/fast so it finishes quickly in limited environments. It records MAE, RMSE, and R² before and after tuning, makes comparison plots, and includes reflection notes.

In [ ]:

# Imports and dataset (built-in)
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
np.random.seed(42)

data = load_diabetes(as_frame=True)
X = data.frame.drop(columns=['target'])
y = data.frame['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print('Data ready.')


In [ ]:

# Baseline model (small forest for speed)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
baseline = RandomForestRegressor(n_estimators=10, random_state=42, n_jobs=1)
baseline.fit(X_train_scaled, y_train)
y_pred_base = baseline.predict(X_test_scaled)
mae_base = mean_absolute_error(y_test, y_pred_base)
rmse_base = mean_squared_error(y_test, y_pred_base, squared=False)
r2_base = r2_score(y_test, y_pred_base)
print('Baseline metrics ->', mae_base, rmse_base, r2_base)
os.makedirs('/mnt/data/outputs', exist_ok=True)
with open('/mnt/data/outputs/baseline_metrics.txt', 'w') as f:
    print('MAE:', mae_base, file=f)
    print('RMSE:', rmse_base, file=f)
    print('R2:', r2_base, file=f)


In [ ]:

# Quick RandomizedSearchCV (very small search for demo)
param_dist = {
    'n_estimators': [10, 20],
    'max_depth': [None, 5],
    'max_features': ['sqrt'],
    'min_samples_split': [2]
}
rf = RandomForestRegressor(random_state=42, n_jobs=1)
rs = RandomizedSearchCV(rf, param_distributions=param_dist, n_iter=2, cv=2, scoring='neg_mean_squared_error', random_state=42, n_jobs=1)
rs.fit(X_train_scaled, y_train)
best = rs.best_estimator_
print('Best params:', rs.best_params_)
with open('/mnt/data/outputs/best_params.txt', 'w') as f:
    print(rs.best_params_, file=f)


In [ ]:

# Evaluate tuned model
y_pred_tuned = best.predict(X_test_scaled)
mae_tuned = mean_absolute_error(y_test, y_pred_tuned)
rmse_tuned = mean_squared_error(y_test, y_pred_tuned, squared=False)
r2_tuned = r2_score(y_test, y_pred_tuned)
print('Tuned metrics ->', mae_tuned, rmse_tuned, r2_tuned)
with open('/mnt/data/outputs/tuned_metrics.txt', 'w') as f:
    print('MAE:', mae_tuned, file=f)
    print('RMSE:', rmse_tuned, file=f)
    print('R2:', r2_tuned, file=f)


In [ ]:

# Plots (MAE, RMSE, R2 before vs after)
import matplotlib.pyplot as plt
metrics_before = [mae_base, rmse_base, r2_base]
metrics_after = [mae_tuned, rmse_tuned, r2_tuned]

plt.figure(figsize=(6,4))
plt.bar(['Before','After'], [metrics_before[0], metrics_after[0]])
plt.title('MAE: Before vs After Tuning')
plt.ylabel('MAE')
plt.tight_layout()
plt.savefig('/mnt/data/outputs/mae_before_after.png')
plt.close()

plt.figure(figsize=(6,4))
plt.bar(['Before','After'], [metrics_before[1], metrics_after[1]])
plt.title('RMSE: Before vs After Tuning')
plt.ylabel('RMSE')
plt.tight_layout()
plt.savefig('/mnt/data/outputs/rmse_before_after.png')
plt.close()

plt.figure(figsize=(6,4))
plt.bar(['Before','After'], [metrics_before[2], metrics_after[2]])
plt.title('R2: Before vs After Tuning')
plt.ylabel('R2')
plt.tight_layout()
plt.savefig('/mnt/data/outputs/r2_before_after.png')
plt.close()

print('Plots saved.')



# Reflection

- Which hyperparameters made the biggest difference?

  See `outputs/best_params.txt` — typically `n_estimators` and `max_depth` change the bias-variance trade-off most.

- Did tuning improve performance or not? Why?

  Check `outputs/tuned_metrics.txt` and `outputs/baseline_metrics.txt`. Smaller tuning runs may or may not improve metrics — a larger search usually helps more.

- Trade-off between accuracy and computation time:

  Larger `n_estimators` and deeper trees improve accuracy but increase training time. Randomized search reduces computation compared to grid search by sampling parameter combos.
